# I/O action

参照透過性が保たれる純粋関数と異なり、キーボードからの入力、画面への出力は「環境の変化に影響される」あるいは「環境を変化させる」という意味において「副作用を持つ」と言う。副作用を持つ関数は「不純 impure」であり、純粋関数 pure functionとは厳密に分離されなければならない。この章では、このような不純な関数の中でも最も多く使われる入出力を取り上げ、純と不純の扱い方について学ぶ。

Haskellにおける入出力はI/Oアクションという形で実現される。つまり、入出力を扱う不純な関数はそれ自体が不純なわけではなく、「I/Oアクションという不純な手続きを返す純粋関数」と言うことができないわけではない。Haskellのソースコードにおいては、`Main`モジュール（暗黙に宣言されている）の中に`main`関数が存在することが期待され、コンパイルされたバイナリはこの`main`をエントリーポイントとして実行される。また、`main`は`:: IO ()`という型を持つ必要があり、バイナリを実行すると実際にはこの`IO`アクションが実行されることになる。

なお、iHaskell notebookでは実行されたコードを`print`関数を利用して表示しているため`IO`アクションそのものの挙動を説明するには不向きであるが、いちいち`.hs`を書くのも面倒だったのでこのままやる。実際には各々のセルの内容を`main = do ...`ブロックの中に記述する必要があることに留意されたい。

In [20]:
putStrLn "Hello, world"
"Hello, world"
print "Hello, world"
:t print

Hello, world

"Hello, world"

"Hello, world"

print :: forall a. Show a => a -> IO ()

IOアクションの結果として入力を受け取る場合、名前の束縛には`<-`を用いる。例えば`:t getLine`は`:: IO String`であり、`:: String`ではない。`<-`により`IO`の「中身」を取り出すことができる。
また、`main`は`:: IO ()`という型からもわかる通り右辺は単一のIOアクションでなければならない。`do`は複数のIOをひとつにまとめる作用がある。

In [21]:
main = do
    putStrLn "What's your name?"
    name <- getLine
    putStrLn $ "Hi, " ++ name

-- in iHaskell notebook, we have to call `main` explicitly
main

What's your name?
Hi, Luke Skywalker

誤って`let ... =`を用いると型エラーとなる。

In [22]:
main = do
    putStrLn "What's your name?"
    let name = getLine
    putStrLn $ "Hi, " ++ name

main

Couldn't match expected type `String' with actual type `IO String'
In the second argument of `(++)', namely `name'
In the second argument of `($)', namely `"Hi, " ++ name'

逆に、静的な（純粋な）値を束縛する場合には`let`でなければならない。

※単に`do`でセルを始めるとmainを宣言する必要がないので、以降そうする。

In [2]:
do
    let name = "Lord Sidious"
    putStrLn $ "As you with, " ++ name

As you with, Lord Sidious

In [3]:
do
    name <- "Master Yoda"
    putStrLn $ "May the force be with you, " ++ name

Couldn't match type `IO' with `[]'
Expected type: [()]
  Actual type: IO ()
In a stmt of a 'do' block: putStrLn $ "May the force be with you, " ++ name
In the expression:
  do { name <- "Master Yoda";
       putStrLn $ "May the force be with you, " ++ name }
In an equation for `it':
    it
      = do { name <- "Master Yoda";
             putStrLn $ "May the force be with you, " ++ name }

Couldn't match expected type `String' with actual type `Char'
In the second argument of `(++)', namely `name'
In the second argument of `($)', namely `"May the force be with you, " ++ name'

### return

Haskellにおけるreturnは、関数の停止とは無縁の単なる`IO`である。`return`は引数として与えられた値を`IO`としてラップして返す。

In [4]:
do
    return 4
    str <- return "Execute order 66"
    putStrLn str

Line 1: Use let
Found:
do return 4
   str <- return "Execute order 66"
   putStrLn str
Why not:
do return 4
   let str = "Execute order 66"
   putStrLn str

Execute order 66

プログラムのflow-controlとして`if`関数を使うことができるが、Haskellの`if`は`else`が必須であるため、`then`節が`IO`を返す場合には`else`節も`IO`を返さなければならない。このようなときに`return`が使える。

In [10]:
do
    line <- getLine
    if null line
        then return ()
        else do
            -- doはネストできる。doは複数のIOをひとつのIOにまとめるものであるから、親のdoからは子のdoが単にひとつのIOとして見える。
            let str = line ++ reverse line
            putStrLn str

Line 3: Use unless
Found:
if null line then return () else
  do let str = line ++ reverse line
     putStrLn str
Why not:
Control.Monad.unless (null line) $
  do let str = line ++ reverse line
     putStrLn str

hogeegoh

### Sequence, mapM, forM

`sequence`は与えられた`IO`のリストのそれぞれを実行し、結果のリストを返す。

In [1]:
sequence [getLine, getLine, getLine]
sequence $ map print [1, 2, 3]

Line 1: Use mapM
Found:
sequence $ map print [1, 2, 3]
Why not:
mapM print [1, 2, 3]

["hoge","fuga","piyo"]

1
2
3
[(),(),()]

この、`sequence $ map`は頻出ボキャブラリであるため、`mapM`という省略記法（というかラッパ関数）が用意されている

In [2]:
mapM print [1, 2, 3]

1
2
3
[(),(),()]

また、iHaskell (ghci) では`IO`の戻り値が`()`だったときにそれを捨てるようになっているが、このようにユニットのリストはユニットと異なるため表示されてしまう。それを避けるのが`mapM_`である。

In [3]:
mapM_ print [1, 2, 3]

1
2
3

`mapM`と引数の順序を逆にしたのが`forM`である。ラムダ式と関数適用演算子を使うことで、いわゆる`foreach`を他の言語で書き慣れたような形式で書ける。

In [8]:
import Control.Monad

forM_ [1, 2, 3] $ \a -> do
    let num = a * 10
    print num

10
20
30